# Task 3
There's a database with the following tables:
* city
    * city_id
    * client_city_id
    * city
    * client_city
* client
    * client_id
    * client_city_id
    * birth_date
    * registration
* promotion
    * promotion_id
    * category_id
    * promotion_name
    * category_name
    * partner_id
    * partner_name
* purchase
    * purchase_id
    * partner_id
    * client_id
    * city_id
    * promotion_id
    * category_id
    * purchase_date
    * price
    * quantity 
    * status

Write a query to get a table containing these fields:
* purchase_date
* purchase_id
* client_id
* client_age
* client_registration_age
* client_category – 'new' if a client purchased just once, otherwise 'old'
* promotion_name
* category_name
* partner_name
* client_city
* city
* revenue
* quantity

This table must contain rows where `status=1` and `purchase_date` is from 01.05.2020 to 01.08.2020

In [4]:
import pandahouse as ph

In [8]:
connection_default = {'host': 'https://clickhouse.lab.karpov.courses',
                      'database':'project_variant_2',
                      'user':'student', 
                      'password':'***'
                     }

In [11]:
task_query = '''
SELECT
    purchase_date,
    purchase_id,
    purchase.client_id AS client_id,
    DATEDIFF(YY, birth_date, now()) AS client_age_years,
    DATEDIFF(DD, registration, now()) AS client_registration_age,
    IF(purchase_date = start_purchase.first_purchase, 'new', 'old') AS client_category,
    promotion_name,
    category_name,
    partner_name,
    city_client.client_city AS client_city,
    uniq_city.city AS city,
    CAST(purchase.price * purchase.quantity AS Float32) AS revenue,
    CAST(purchase.quantity AS Int8) AS quantity 
FROM
    project_variant_2.purchase 
    LEFT JOIN
        project_variant_2.client 
        ON purchase.client_id = client.client_id 
    LEFT JOIN
        project_variant_2.promotion 
        ON purchase.promotion_id = promotion.promotion_id 
    LEFT JOIN
        (
            SELECT DISTINCT
                client_city_id,
                client_city 
            FROM
                project_variant_2.city
        )
        AS city_client 
        ON client.client_city_id = city_client.client_city_id 
    LEFT JOIN
        (
            SELECT DISTINCT
                city_id,
                city 
            FROM
                project_variant_2.city
        )
        AS uniq_city 
        ON purchase.city_id = uniq_city.city_id 
    LEFT JOIN
        (
            SELECT
                client_id,
                MIN(purchase_date) AS first_purchase 
            FROM
                project_variant_2.purchase 
            GROUP BY
                client_id
        )
        AS start_purchase 
        ON purchase.client_id = start_purchase.client_id 
WHERE
    status = 1 
    AND purchase_date BETWEEN '2020-05-01' AND '2020-08-01'
'''

In [12]:
query_result = ph.read_clickhouse(query=task_query, connection=connection_default)

In [13]:
query_result

,purchase_date,purchase_id,client_id,client_age_years,client_registration_age,client_category,promotion_name,category_name,partner_name,client_city,city,revenue,quantity
0,2020-06-05,0,853,29,794,old,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Ярославль,Киев,150.0,5
1,2020-06-15,1,406,24,669,old,campaign_4,Прочее,KarpovCourses,Минск,Новгород,1150.0,5
2,2020-08-01,9,867,25,432,old,campaign_1,Бакалея,Google,Киев,Москва,1840.0,4
3,2020-07-16,15,109,30,995,old,campaign_4,Прочее,KarpovCourses,Щучинщина,Санкт-Петербург,440.0,8
4,2020-06-29,22,528,21,495,old,campaign_2,Фрукты,"ООО Ширяева, Хохлова и Тимофеева",Щучинщина,Щучинщина,390.0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2730,2020-07-28,8241,602,23,639,old,campaign_1,Бакалея,Google,Щучинщина,Вена,780.0,6
2731,2020-07-15,8245,46,31,926,old,campaign_1,Бакалея,Google,Москва,Минск,2275.0,5
2732,2020-06-09,8246,954,25,505,old,campaign_4,Прочее,KarpovCourses,Санкт-Петербург,Вена,3120.0,8
2733,2020-08-01,8247,279,22,645,old,campaign_4,Прочее,KarpovCourses,Казань,Новгород,735.0,7
